# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [38]:
# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)

root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: date (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: date (nullable = true)
 |-- EXPIRED: date (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: date (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (nullable

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [39]:
filtered_df = df.filter((col('SALARY_FROM').isNotNull()) & (col('SALARY_FROM') > 0))

In [40]:
salary_by_employment_type = filtered_df.groupBy('EMPLOYMENT_TYPE_NAME').agg(
    F.min('SALARY_FROM').alias('min_salary'),
    F.max('SALARY_FROM').alias('max_salary'),
    F.avg('SALARY_FROM').alias('avg_salary'),
    F.expr('percentile_approx(SALARY_FROM, 0.5)').alias('median_salary')
)
salary_by_employment_type.show(truncate = False)

+----------------------+----------+----------+-----------------+-------------+
|EMPLOYMENT_TYPE_NAME  |min_salary|max_salary|avg_salary       |median_salary|
+----------------------+----------+----------+-----------------+-------------+
|Part-time / full-time |15600     |455375    |81574.44254658386|72000        |
|Part-time (≤ 32 hours)|12000     |500000    |74000.37215189873|65000        |
|Full-time (> 32 hours)|10230     |800000    |95042.60378037216|89990        |
+----------------------+----------+----------+-----------------+-------------+



In [41]:
df_pandas = filtered_df.select('EMPLOYMENT_TYPE_NAME', 'SALARY_FROM').toPandas()

fig = px.box(df_pandas, 
             x = 'EMPLOYMENT_TYPE_NAME', 
             y = 'SALARY_FROM', 
             title = 'Salary Distribution by Employment Type',
             labels = {'EMPLOYMENT_TYPE_NAME': 'Employment Type', 'SALARY_FROM': 'Salary'},
             color = 'EMPLOYMENT_TYPE_NAME', 
             template= "simple_white",
             color_discrete_sequence = px.colors.qualitative.Set2)

fig.update_layout(
    font = dict(family = 'Arial', size = 16),
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2],
        ticktext = ['Mixed', 'Full-Time (> 32 hr)', 'Part-Time (≤ 32 hr)']),
    yaxis_title = 'Salary',
    plot_bgcolor = 'white',
    paper_bgcolor = 'lightgray'
)

fig.show()

The box plot shows salary distributions across different employment types. It highlights that Full-Time jobs have the highest median salary and a wider interquartile range (IQR), indicating higher salaries and greater variability in earnings. The presence of outliers suggests that some positions in this category offer exceptionally high salaries. In addition, Part-Time salaries are generally lower, with a lower median and a tighter IQR, indicating less variability. Finally, Mixed employment type falls between the two, showing moderate salary dispersion. This analysis suggests that full-time jobs generally offer higher pay than part-time roles, but salaries within each category still vary significantly.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [44]:
salary_by_NAICS = filtered_df.groupBy('NAICS2_NAME').agg(
    F.min('SALARY_FROM').alias('min_salary'),
    F.max('SALARY_FROM').alias('max_salary'),
    F.avg('SALARY_FROM').alias('avg_salary'),
    F.expr('percentile_approx(SALARY_FROM, 0.5)').alias('median_salary')
)
salary_by_NAICS.show(truncate = False)

+------------------------------------------------------------------------+----------+----------+------------------+-------------+
|NAICS2_NAME                                                             |min_salary|max_salary|avg_salary        |median_salary|
+------------------------------------------------------------------------+----------+----------+------------------+-------------+
|Administrative and Support and Waste Management and Remediation Services|12480     |800000    |92487.35049504951 |84156        |
|Public Administration                                                   |10230     |211300    |69426.60515603799 |65604        |
|Real Estate and Rental and Leasing                                      |10230     |260000    |79555.65198237885 |70000        |
|Information                                                             |11148     |500000    |112628.10814249364|105000       |
|Unclassified Industry                                                   |10800     |70000

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 3rd question here

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 4th question here

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 5th question here

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
df_pandas = filtered_df.select("NAICS2_NAME", "SALARY_FROM").toPandas()


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 7th question here

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 8th question here